In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# For EQUITY class
# Calculates intrabucket capital charge for all buckets for three correlation scenario acc. to FRTB 

class Buckets:
    def __init__(self, *net_sensitivities, risk_weight, base_correlation):
        self.net_sensitivities = net_sensitivities
        self.RW = risk_weight
        self.correlation = base_correlation

    def weighted_net_sensitivity(self):
        return self.RW * np.array(self.net_sensitivities)
    
    def calculate_WSk_sum(self):
        WSk_matrix = self.weighted_net_sensitivity()
        return np.sum(WSk_matrix)
    
    def calculate_intra_bucket_capital_charge(self):
        scenarios = ['Base', 'High', 'Low']
        results = []

        for scenario in scenarios:
            WSk_matrix = np.array([self.weighted_net_sensitivity()])
            correlation_matrix = self.IntraBucketCorrelation(scenario)
            capital_charge = np.sqrt(WSk_matrix @ correlation_matrix @ WSk_matrix.T)
            results.append((scenario, capital_charge))

        return results
    
    
    
    def IntraBucketCorrelation(self, correlation_scenario):
        values = len(self.net_sensitivities)
        rho_matrix = np.full((values, values), self.correlation)
        np.fill_diagonal(rho_matrix, 1.0)

        if correlation_scenario == 'High':
            rho_matrix = np.full((values, values), min(self.correlation * 1.25, 1))
            np.fill_diagonal(rho_matrix, 1.0)
            return rho_matrix
        elif correlation_scenario == 'Low':
            rho_matrix = np.full((values, values), max(self.correlation * 2 - 1, 0.75 * self.correlation))
            np.fill_diagonal(rho_matrix, 1.0)
            return rho_matrix

        return rho_matrix

class Bucket_1(Buckets):
    pass

class Bucket_2(Buckets):
    pass

class Bucket_3(Buckets):
    pass

class Bucket_4(Buckets):
    pass

class Bucket_5(Buckets):
    pass

class Bucket_6(Buckets):
    pass

class Bucket_7(Buckets):
    pass

class Bucket_8(Buckets):
    pass

class Bucket_9(Buckets):
    pass

class Bucket_10(Buckets):
    pass

class Bucket_11(Buckets):
    pass

class Bucket_12(Buckets):
    pass

class Bucket_13(Buckets):
    pass


    
# Data for different buckets required from user as inputs
# Bucket number -> delta net sensitivities -> risk weights -> correlations(below data is in this order)
bucket_data = [
    (Bucket_1,[270.909,36.3636],0.55,0.15),
(Bucket_2, [23.33, -15, 20], 0.60, 0.15),
(Bucket_3, [60,240], 0.45, 0.15),
(Bucket_4, [-40], 0.55, 0.15),
(Bucket_5, [-30,480], 0.30, 0.25),
(Bucket_6, [54.285], 0.35, 0.25),
(Bucket_7, [370,30], 0.40, 0.25),
(Bucket_8, [230, 50, 25], 0.50, 0.25),
(Bucket_9, [8.57], 0.70, 0.075),
(Bucket_10, [112], 0.50, 0.125),
(Bucket_11, [92.857], 0.70, 0.00),
(Bucket_12, [0.00], 0.15, 0.80),
(Bucket_13, [352], 0.25, 0.80)
]

# Create instances for all buckets
buckets = [bucket_class(*net_sensitivities, risk_weight=risk_weight, base_correlation=base_correlation) 
           for bucket_class, net_sensitivities, risk_weight, base_correlation in bucket_data]

results_df = pd.DataFrame(index=range(1, len(buckets) + 1), columns=['Base', 'High', 'Low'])

# Calculate intra-bucket capital charges and populate the DataFrame
for bucket_index, bucket in enumerate(buckets, start=1):
    capital_charges = bucket.calculate_intra_bucket_capital_charge()
    
    wsk_sum = bucket.calculate_WSk_sum()
    results_df.loc[bucket_index, 'WSk Sum/ Sb'] = wsk_sum
    
    for scenario, Kb in capital_charges:
        results_df.loc[bucket_index, scenario] = Kb[0, 0]

# Print the results DataFrame
print("Intrabucket Capital Charges:")
results_df

Intrabucket Capital Charges:


,Base,High,Low,WSk Sum/ Sb
1,153.280733,154.008061,152.549937,168.99993
2,20.028535,19.904566,20.151741,16.99800
3,115.185937,116.131391,114.232657,135.00000
4,22.0,22.0,22.0,-22.00000
5,142.017605,141.446103,142.586816,135.00000
6,18.99975,18.99975,18.99975,18.99975
7,151.44636,152.177528,150.711645,160.00000
8,127.744863,129.98798,125.461648,152.50000
9,5.999,5.999,5.999,5.99900
10,56.0,56.0,56.0,56.00000


In [3]:
# clculates sum of squared capital charges for each bucket for each scenario acc. to FRTB 
# this is basically the sum of Kb**2 term used in final calculation of delta risk charge for equity class

def calculate_sum_squared(dataframe, columns):
    return np.sum(dataframe[columns] ** 2)

# List of columns for which i want to calculate sum of squared values
columns = ['Base', 'High', 'Low']

sum_squared_values = calculate_sum_squared(results_df, columns)

print("Sum of squared values for each column:", sum_squared_values)

Sum of squared values for each column: Base    112573.640811
High    113649.015063
Low      111498.26656
dtype: object


In [4]:
# creates interbucket correlation matrix for all three correlation scenario
# Used in final calculation of equity delta risk charge 

def interbucket_correlation(column_names, index_names, correlation_scenario):
    num_buckets = len(column_names)
    correlation_matrix = np.zeros((num_buckets, num_buckets))

    for i, col_name in enumerate(column_names):
        for j, idx_name in enumerate(index_names):
            if col_name == idx_name:
                correlation_matrix[i, j] = 0.00  # 0% correlation for diagonal elements
           
            elif col_name in ['Bucket 11'] or idx_name in ['Bucket 11']:
                correlation_matrix[i, j] = 0.00  # 0% correlation for pairs involving Bucket 11
            
            elif col_name in ['Bucket 12', 'Bucket 13'] and idx_name in ['Bucket 12', 'Bucket 13']:
                correlation_matrix[i, j] = 0.75   # 75% correlation for pairs involving Bucket 12 and 13
            
            elif col_name in ['Bucket 12', 'Bucket 13'] or idx_name in ['Bucket 12', 'Bucket 13']:
                correlation_matrix[i, j] = 0.45    # 45% correlation for pairs involving Bucket 12 or 13
             
            else:
                correlation_matrix[i, j] = 0.15  # 15% correlation for other pairs

    if correlation_scenario == 'High':
        gamma_matrix = np.minimum(correlation_matrix * 1.25, 1)
    elif correlation_scenario == 'Low':
        gamma_matrix = np.maximum(correlation_matrix * 2 - 1, 0.75 * correlation_matrix)
    else:
        gamma_matrix = correlation_matrix
    
    np.fill_diagonal(gamma_matrix, 0.00)
    return pd.DataFrame(gamma_matrix, columns=index_names, index=column_names)

# Bucket names
bucket_names = ['Bucket 1', 'Bucket 2', 'Bucket 3', 'Bucket 4', 'Bucket 5', 'Bucket 6', 'Bucket 7',
                'Bucket 8', 'Bucket 9', 'Bucket 10', 'Bucket 11', 'Bucket 12', 'Bucket 13']

# Create the interbucket correlation matrix for different scenarios
correlation_matrix_base = interbucket_correlation(bucket_names, bucket_names, 'Base')
correlation_matrix_high = interbucket_correlation(bucket_names, bucket_names, 'High')
correlation_matrix_low = interbucket_correlation(bucket_names, bucket_names, 'Low')
print("Interbucket correlation matrix")
correlation_matrix_base

Interbucket correlation matrix


,Bucket 1,Bucket 2,Bucket 3,Bucket 4,Bucket 5,Bucket 6,Bucket 7,Bucket 8,Bucket 9,Bucket 10,Bucket 11,Bucket 12,Bucket 13
Bucket 1,0.00,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 2,0.15,0.00,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 3,0.15,0.15,0.00,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 4,0.15,0.15,0.15,0.00,0.15,0.15,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 5,0.15,0.15,0.15,0.15,0.00,0.15,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 6,0.15,0.15,0.15,0.15,0.15,0.00,0.15,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 7,0.15,0.15,0.15,0.15,0.15,0.15,0.00,0.15,0.15,0.15,0.0,0.45,0.45
Bucket 8,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.00,0.15,0.15,0.0,0.45,0.45
Bucket 9,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.00,0.15,0.0,0.45,0.45
Bucket 10,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.00,0.0,0.45,0.45


In [5]:
# final function to calculate delta risk charge for Equity class

def CalculateEquityDeltaCharge(dataframe,correlation_scenario = 'Base'):
    wsk_sum_column = results_df['WSk Sum/ Sb'].values
   # Convert the numpy array to a column matrix (nx1 matrix)
    wsk_sum_matrix = wsk_sum_column.reshape(-1, 1)
    
    
    if correlation_scenario == 'High':
        squared_kb_high = calculate_sum_squared(results_df, 'High')
        capital_high =  np.sqrt(squared_kb_high + (wsk_sum_matrix.T@correlation_matrix_high@wsk_sum_matrix))
        return capital_high
    
    elif  correlation_scenario == 'Low':
        squared_kb_low = calculate_sum_squared(results_df, 'Low')
        capital_low =  np.sqrt(squared_kb_low + (wsk_sum_matrix.T@correlation_matrix_low@wsk_sum_matrix))
        return capital_low
    
    else:
        squared_kb_base = calculate_sum_squared(results_df, 'Base')
        capital_base =  np.sqrt(squared_kb_base + (wsk_sum_matrix.T@correlation_matrix_base@wsk_sum_matrix))
        return capital_base
    
    
        
# Final delta risk figure is the maximum of three correlation scemnarios
K_base = CalculateEquityDeltaCharge(results_df,correlation_scenario = 'Base')
K_high = CalculateEquityDeltaCharge(results_df,correlation_scenario = 'High')
K_low = CalculateEquityDeltaCharge(results_df,correlation_scenario = 'Low')
Equity_Delta_risk_capital = max(np.array([K_base,K_high,K_low]))
print('Delta risk capital charge for Equity class:')
print(Equity_Delta_risk_capital)


Delta risk capital charge for Equity class:
[[549.3673542]]


In [ ]:
import numpy as np
import pandas as pd

class Bucket:
    def __init__(self, name, net_sensitivities, risk_weight, base_correlation):
        self.name = name
        self.net_sensitivities = net_sensitivities
        self.RW = risk_weight
        self.base_correlation = base_correlation

    def weighted_net_sensitivity(self):
        return self.RW * np.array(self.net_sensitivities)

    def calculate_WSk_sum(self):
        return np.sum(self.weighted_net_sensitivity())

    def calculate_intra_bucket_capital_charge(self):
        scenarios = ['Base', 'High', 'Low']
        results = []

        for scenario in scenarios:
            WSk_matrix = np.array([self.weighted_net_sensitivity()])
            correlation_matrix = self.IntraBucketCorrelation(scenario)
            capital_charge = np.sqrt(WSk_matrix @ correlation_matrix @ WSk_matrix.T)
            results.append((scenario, capital_charge[0, 0]))

        return results

    def IntraBucketCorrelation(self, correlation_scenario):
        values = len(self.net_sensitivities)
        rho_matrix = np.full((values, values), self.base_correlation)
        np.fill_diagonal(rho_matrix, 1.0)

        if correlation_scenario == 'High':
            rho_matrix = np.full((values, values), min(self.base_correlation * 1.25, 1))
            np.fill_diagonal(rho_matrix, 1.0)
        elif correlation_scenario == 'Low':
            rho_matrix = np.full((values, values), max(self.base_correlation * 2 - 1, 0.75 * self.base_correlation))
            np.fill_diagonal(rho_matrix, 1.0)

        return rho_matrix

# Bucket data as a list of dictionaries
bucket_data = [
    {'name': 'Bucket_1', 'net_sensitivities': [270.909, 36.3636], 'risk_weight': 0.55, 'base_correlation': 0.15},
    {'name': 'Bucket_2', 'net_sensitivities': [23.33, -15, 20], 'risk_weight': 0.60, 'base_correlation': 0.15},
    {'name': 'Bucket_3', 'net_sensitivities': [60, 240], 'risk_weight': 0.45, 'base_correlation': 0.15},
    {'name': 'Bucket_4', 'net_sensitivities': [-40], 'risk_weight': 0.55, 'base_correlation': 0.15},
    {'name': 'Bucket_5', 'net_sensitivities': [-30, 480], 'risk_weight': 0.30, 'base_correlation': 0.25},
    {'name': 'Bucket_6', 'net_sensitivities': [54.285], 'risk_weight': 0.35, 'base_correlation': 0.25},
    {'name': 'Bucket_7', 'net_sensitivities': [370, 30], 'risk_weight': 0.40, 'base_correlation': 0.25},
    {'name': 'Bucket_8', 'net_sensitivities': [230, 50, 25], 'risk_weight': 0.50, 'base_correlation': 0.25},
    {'name': 'Bucket_9', 'net_sensitivities': [8.57], 'risk_weight': 0.70, 'base_correlation': 0.075},
    {'name': 'Bucket_10', 'net_sensitivities': [112], 'risk_weight': 0.50, 'base_correlation': 0.125},
    {'name': 'Bucket_11', 'net_sensitivities': [92.857], 'risk_weight': 0.70, 'base_correlation': 0.00},
    {'name': 'Bucket_12', 'net_sensitivities': [0.00], 'risk_weight': 0.15, 'base_correlation': 0.80},
    {'name': 'Bucket_13', 'net_sensitivities': [352], 'risk_weight': 0.25, 'base_correlation': 0.80},
]

buckets = []
for data in bucket_data:
    bucket = Bucket(**data)
    buckets.append(bucket)

results_df = pd.DataFrame(index=[bucket.name for bucket in buckets], columns=['Base', 'High', 'Low', 'WSk Sum/ Sb'])

# Calculate intra-bucket capital charges and populate the DataFrame
for bucket in buckets:
    capital_charges = bucket.calculate_intra_bucket_capital_charge()
    wsk_sum = bucket.calculate_WSk_sum()
    
    results_df.loc[bucket.name, 'WSk Sum/ Sb'] = wsk_sum
    
    for scenario, Kb in capital_charges:
        results_df.loc[bucket.name, scenario] = Kb

# Print the results DataFrame
print("Intrabucket Capital Charges:")
results_df
